# GPX Export for Outdooractive

This notebook demonstrates how to:
1. Select a rectangular area on a map of Norway
2. Filter trails that intersect with the selected area
3. Export the trails to GPX format suitable for Outdooractive import

## 1. Setup and Data Loading

In [ ]:
import sys
from pathlib import Path

import geopandas as gpd

# Add project root to path
project_root = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Import our modules
from trails.io.export.area_selector import (  # noqa: E402
    create_interactive_selection_map,
    save_selection_preset,
)
from trails.io.export.gpx import export_to_gpx, filter_trails_by_bbox  # noqa: E402
from trails.io.sources.geonorge import Language, Source  # noqa: E402

print("✓ Modules loaded")

In [ ]:
# Load trail data (using cache in repo root)
print("Loading trail data...")
source = Source(cache_dir=str(project_root / ".cache"))
trail_data = source.load_turrutebasen(language=Language.EN)

# Get the main trail layer
trails = trail_data.spatial_layers["hiking_trail_centerline"]
print(f"\nLoaded {len(trails):,} trail segments")
print(f"CRS: {trails.crs}")
print(f"Total bounds: {trails.total_bounds}")

## 2. Interactive Area Selection

Use the map below to select your area of interest:

## 2. Interactive Area Selection (NEW!)

### ✨ Automatic Bounds Capture

1. **Search for a city** using the magnifying glass icon
2. **Draw a rectangle** around your area of interest
3. **Bounds are captured automatically** - no copying needed!

The selected bounds will be available in the next cell.

In [ ]:
# Create interactive selection map with automatic bounds capture
map_widget, bounds_container = create_interactive_selection_map(trails, zoom_start=5)

# Display the interactive map
map_widget

## 3. Save Area Preset (Optional)

Save your selected area for future use:

In [ ]:
# Save the current selection as a preset
selected_bounds = bounds_container["bounds"]

if selected_bounds:
    preset_file = project_root / ".cache" / "area_presets.json"
    save_selection_preset("gjerstad_nissedal", selected_bounds, "Gjerstad and Nissedal area", presets_file=preset_file)
    print("✓ Area preset saved!")

    # Load it back (example)
    from trails.io.export.area_selector import load_selection_preset

    loaded_bounds = load_selection_preset("gjerstad_nissedal", presets_file=preset_file)
    print(f"\nLoaded preset: {loaded_bounds}")

## 4. Process Selected Area

In [ ]:
# Get the automatically captured bounds
selected_bounds = bounds_container["bounds"]

if selected_bounds:
    print(f"✅ Using auto-captured bounds: {selected_bounds}")
    print(f"  Longitude: {selected_bounds[0]:.2f} to {selected_bounds[2]:.2f}")
    print(f"  Latitude: {selected_bounds[1]:.2f} to {selected_bounds[3]:.2f}")
else:
    print("⚠️ No area selected yet! Please draw a rectangle on the map above.")
    print("\nAlternatively, you can manually set bounds:")
    print("# selected_bounds = (8.5, 58.8, 9.2, 59.2)  # Gjerstad/Nissedal")
    print("# selected_bounds = (10.3, 59.8, 11.0, 60.2)  # Oslo")
    print("# selected_bounds = (5.0, 60.2, 5.8, 60.6)  # Bergen")

    # Default to Gjerstad/Nissedal for demo
    selected_bounds = (8.5, 58.8, 9.2, 59.2)
    print("\n📍 Using default area: Gjerstad/Nissedal")

In [ ]:
# Filter trails by selected area
if selected_bounds:
    # Convert bounds to trail CRS
    from shapely.geometry import box

    bounds_gdf = gpd.GeoDataFrame([1], geometry=[box(*selected_bounds)], crs="EPSG:4326")

    if trails.crs != "EPSG:4326":
        bounds_gdf = bounds_gdf.to_crs(trails.crs)  # type: ignore[arg-type]
        area_bounds = bounds_gdf.geometry[0].bounds
    else:
        area_bounds = selected_bounds

    # Filter with optional buffer
    buffer_km = 5  # Add 5km buffer around selection
    filtered_trails = filter_trails_by_bbox(trails, area_bounds, buffer_m=buffer_km * 1000)

    print("\n📊 Filtering results:")
    print(f"  Original trails: {len(trails):,}")
    print(f"  Filtered trails: {len(filtered_trails):,}")
    print(f"  Buffer applied: {buffer_km} km")

    # Calculate statistics
    total_length_km = filtered_trails.geometry.length.sum() / 1000
    print(f"  Total length: {total_length_km:,.1f} km")

## 4. Enrich Trail Data

Add names and metadata from attribute tables:

In [ ]:
# Join with attribute data
hiking_attrs = trail_data.attribute_tables["hiking_trail_info_table"]

# Join filtered trails with attributes
enriched_trails = filtered_trails.merge(
    hiking_attrs[
        [
            "hiking_trail_fk",
            "trail_number",
            "trail_name",
            "special_hiking_trail_type",
            "maintenance_responsible",
            "difficulty",
        ]
    ],
    left_on="local_id",
    right_on="hiking_trail_fk",
    how="left",
)

print("\n📝 Trail data enrichment:")
print(f"  Trails with numbers: {enriched_trails['trail_number'].notna().sum():,}")
print(f"  Trails with names: {enriched_trails['trail_name'].notna().sum():,}")
print(f"  Trails with types: {enriched_trails['special_hiking_trail_type'].notna().sum():,}")
print(f"  Trails with maintainer: {enriched_trails['maintenance_responsible'].notna().sum():,}")
print(f"  Trails with difficulty: {enriched_trails['difficulty'].notna().sum():,}")

# Show sample
print("\n Sample trails:")
sample = enriched_trails[enriched_trails["trail_name"].notna()].head(5)
for _, trail in sample.iterrows():
    print(f"  • {trail['trail_name']}: {trail.geometry.length / 1000:.1f} km")

## 5. Export to GPX

Generate GPX file optimized for Outdooractive:

In [ ]:
# Export options
output_dir = project_root / "output"
output_dir.mkdir(exist_ok=True)
gpx_file = output_dir / "trails_export.gpx"
print("🔧 Export settings:")
print(f"  Output file: {gpx_file}")
print("  Simplification: Enabled (reduces file size)")
print(f"  Max trails: None (export all {len(enriched_trails)} trails)")
print("  Description fields: maintenance, difficulty, marking")

# Perform export
output_path, stats = export_to_gpx(
    enriched_trails,
    gpx_file,
    name_field="trail_name",
    desc_fields=[
        "trail_number",
        "special_hiking_trail_type",
        "difficulty",
        "marking",
        "season",
        "trail_follows",
        "maintenance_responsible",
        "accuracy",
        "data_capture_date",
        "update_date",
    ],
    simplify_tolerance=0.00001,  # ~1 meter at Norway latitudes
    max_trails=None,  # Set to e.g. 100 for testing
)

print("\n✅ Export complete!")
print("\n📊 Export statistics:")
for key, value in stats.items():
    if key == "file_size_mb":
        print(f"  {key}: {value:.2f} MB")
    else:
        print(f"  {key}: {value:,}")

## 7. Verify GPX File

Quick validation of the generated GPX:

In [ ]:
# Parse and validate GPX
from lxml import etree

if gpx_file.exists():
    # Parse GPX
    with open(gpx_file, "rb") as f:
        tree = etree.parse(f)
        root = tree.getroot()

    # Count elements
    ns = {"gpx": "http://www.topografix.com/GPX/1/1"}
    tracks = root.xpath("//gpx:trk", namespaces=ns)
    segments = root.xpath("//gpx:trkseg", namespaces=ns)
    points = root.xpath("//gpx:trkpt", namespaces=ns)

    print("📋 GPX file validation:")
    print(f"  Tracks: {len(tracks)}")
    print(f"  Segments: {len(segments)}")
    print(f"  Points: {len(points):,}")
    print(f"  Avg points/track: {len(points) / len(tracks):.0f}" if tracks else "")

    # Show first track
    if tracks:
        first_track = tracks[0]
        name = first_track.find("gpx:name", ns)
        desc = first_track.find("gpx:desc", ns)
        print("\nFirst track:")
        print(f"  Name: {name.text if name is not None else 'No name'}")
        print(f"  Description: {desc.text[:100] if desc is not None else 'No description'}...")

    print("\n✅ GPX file is valid and ready for Outdooractive import!")
    print("\n📱 Next steps:")
    print(f"   1. Open {gpx_file}")
    print("   2. Upload to Outdooractive.com")
    print("   3. Or import via Outdooractive mobile app")
else:
    print("❌ GPX file not found")

## Summary

### What We've Accomplished:

1. **Area Selection**: Interactive map with rectangle drawing tool
2. **Spatial Filtering**: Efficient filtering of trails by bounding box with optional buffer
3. **GPX Export**: Standards-compliant GPX 1.1 format with:
   - Trail names and descriptions
   - Geometry simplification to reduce file size
   - Proper WGS84 coordinates
4. **Outdooractive Compatibility**: 
   - Each trail as separate `<trk>` element
   - Metadata in description fields
   - Optimized point density

### Limitations & Considerations:

- **File Size**: Large areas may produce files >20MB (Outdooractive limit)
- **Point Density**: Simplified to ~1m accuracy to reduce size
- **Metadata**: Limited to available fields in Norwegian dataset
- **Interactive Selection**: Full interactivity requires opening HTML file in browser

### Possible Enhancements:

1. Add elevation data from DEM files
2. Split large exports into multiple GPX files
3. Filter by trail difficulty or type
4. Add waypoints for trail intersections
5. Generate route suggestions based on difficulty